Create the llm

In [ ]:
import os
from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model="llama3-70b-8192"
)



Google api key

In [ ]:
##key = os.getenv("GOOGLE_API_KEY")

Define Tools

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults


web_search_tool = TavilySearchResults(description="Procura na web por informações sobre o termo informado. Sempre utilize esta ferramenta quando o assunto em questão for sobre uma tecnologia.", max_results=5)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("./data/Base.pdf", extract_images=True)
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, HuggingFaceEmbeddings())
retriever = vector.as_retriever()

In [ ]:
from langchain.tools.retriever import create_retriever_tool

rag_search_tool = create_retriever_tool(
    retriever,
    "busca_informacoes",
    "Procura informações sobre a sua empresa. Dentre essas informações estão os valores da empresa, habilidades comportamentais, organização interna e outros aspectos da sua cultura organizacional."
)

In [ ]:
from calendar_tools import GetCalendarEventsTool, CreateCalendarEventTool, CurrentTimeTool, TimeDeltaTool, SpecificTimeTool

calendar_tool = [GetCalendarEventsTool(), CreateCalendarEventTool(), CurrentTimeTool(), TimeDeltaTool(), SpecificTimeTool()]

In [ ]:
tools_search = [rag_search_tool, web_search_tool]

Create the agent

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_helper = ChatPromptTemplate.from_messages([
  ("system", "Você é um desenvolvedor sênior cujo objetivo é facilitar a integração de novos funcionários na sua empresa, ajudando-os a se familiarizar rapidamente com a cultura, políticas, programas e ferramentas de trabalho. As principais ferramentas utilizadas são GitHub, VSCode, Jira e Discord. Também pode usar qualquer outra ferramenta que interaja com essas. Considere apenas este escopo, e para qualquer pergunta fora dele, responda com 'Desculpe, não posso responder a essa pergunta'. Seja sempre educado, forneça respostas diretas, com guias passo a passo quando necessário, e em português."),
  ("placeholder", "{chat_history}"),
  ("human", "{input}"),
  ("placeholder", "{agent_scratchpad}"),
])

In [ ]:
prompt_revisor = ChatPromptTemplate.from_messages([
  ("system", "Você é um verificador de conteúdo. Sua função é analisar a resposta gerada pelo primeiro agente e garantir que ela não contenha referências a outras empresas que não seja a nossa (tech4.ai), informações sensiveis ou discurso de ódio. Se a resposta estiver adequada, confirme com 'Resposta válida'. Se houver problemas, rejeite a resposta, retornando apenas 'Desculpe, essa informação está fora de minhas capacidades' e mais nenhum texto."),
  ("placeholder", "{chat_history}"),
  ("human", "{input}"),
  ("placeholder", "{agent_scratchpad}"),
])

In [ ]:
prompt_request_identify = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente inteligente especializado em entender e classificar requisições de usuários em uma empresa de tecnologia. Abaixo está uma solicitação de um usuário. Sua tarefa é identificar se a solicitação é uma dúvida geral que deve ser respondida pelo agente de ajuda ou se é uma requisição relacionada ao calendário, como agendamento ou consulta de reuniões. Responda apenas com 'Calendário' se a requisição for relacionada ao calendário ou 'Ajuda' se for uma dúvida geral. Dúvidas do tipo 'como o calendário funciona' devem ser respondidas pelo agente de ajuda. Considere o contexto da conversa e responda de acordo."),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [ ]:


prompt_calendar = ChatPromptTemplate.from_messages([
    ("system", """
    Você é um agente responsável por gerenciar o Google Calendar. Sua tarefa é auxiliar na marcação de reuniões, consulta de eventos e fornecimento de detalhes sobre eventos. Utilize sempre a data e a hora atuais como referência. Abaixo estão exemplos de solicitações que você pode receber e como deve respondê-las:

    Contexto Atual:
    - Horário atual: {current_datetime}
    - Dia da semana: {current_weekday}

    Exemplos de Soli:citações e Respostas
    1. Solicitação: "Marque uma reunião com João e Maria amanhã às 15h."
    Resposta: "Entendido, vou marcar uma reunião com João e Maria amanhã às 15h."
    2. Solicitação: "Quais reuniões tenho para amanhã?"
    Resposta: "Você tem uma reunião de equipe às 10h, na Sala de Conferências 1, com 3 participantes, e uma revisão de projeto às 14h, na Sala de Reuniões 2, com 5 participantes."
    3. Solicitação: "Forneça os detalhes da reunião com o cliente na próxima terça-feira."
    Resposta: "A reunião com o cliente está agendada para a próxima terça-feira às 11h. O local é a Sala de Conferências 1 e o objetivo é discutir os requisitos do projeto."
    4. Solicitação: "Marque uma reunião com o cliente na próxima quinta-feira às 14h."
    Resposta: "Nesta data e horário, você já tem uma reunião agendada. Escolha outro horário ou data para a reunião com o cliente."
        
    Diretrizes Adicionais:
    - Respostas: Não faça perguntas ao usuário, apenas forneça as informações solicitadas e feedbacks claros sobre a solicitação.
    - Agendamento Futuro: Agende reuniões apenas para datas futuras.
    - Referência Temporal: Considere termos como "amanhã" ou dias da semana mencionados, referindo-se sempre ao próximo dia correspondente.
    - Formato de Data e Hora: Converta todas as datas e horas para o formato RFC 3339 (exemplo: 2022-01-01T15:00:00).
    - Informações Necessárias: Para agendar uma reunião, o usuário deve fornecer:
    - Assunto da reunião
    - Data
    - Hora
    Caso alguma dessas informações não seja fornecida, solicite ao usuário que reformule a solicitação incluindo todos os detalhes.

    - Solicitações Incompreensíveis: Se você não entender uma solicitação, peça ao usuário para reformulá-la.
    """
    ),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [ ]:
from langchain.agents import create_tool_calling_agent

helper = create_tool_calling_agent(llm, tools_search, prompt_helper)
verify = create_tool_calling_agent(llm, [], prompt_revisor)
request_identify = create_tool_calling_agent(llm, [], prompt_request_identify)
calendar = create_tool_calling_agent(llm, calendar_tool, prompt_calendar)

In [ ]:
from langchain.agents import AgentExecutor

agent_helper = AgentExecutor(agent=helper, tools=tools_search, verbose=True)
agent_verifier = AgentExecutor(agent=verify, tools=[], verbose=True)
agent_request_identifier = AgentExecutor(agent=request_identify, tools=[], verbose=True)
agent_calendar = AgentExecutor(agent=calendar, tools=calendar_tool, verbose=True)

Config

In [ ]:
config = {"configurable": {"session_id": "<foo>"}}

Run the agent

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import datetime

current_datetime = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
current_weekday = datetime.datetime.now().strftime("%A")

message_history = ChatMessageHistory()
agent_helper_with_chat_history = RunnableWithMessageHistory(
    agent_helper,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

agent_google_calendar_with_chat_history = RunnableWithMessageHistory(
    agent_calendar,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    current_datetime=current_datetime,
    current_weekday=current_weekday,
)

def verify_response(input: str, response: str) -> str:
    verifier_input = f"Verifique a seguinte resposta: '{response}'"
    verification_result = RunnableWithMessageHistory(
        agent_verifier,
        input_messages_key=verifier_input,
        history_messages_key="chat_history",
    )
    return verification_result["output"]

In [ ]:
    # def verify_response(input: str, response: str) -> str:
    #     verifier_input = f"Verifique a seguinte resposta: '{response}'"
    #     verification_result = agent_verifier.invoke(
    #         {"input": verifier_input},
    #         config=config,
    #     )
    #     return verification_result["output"]

In [ ]:
def identify_request(input: str) -> str:
    identify_input = f"Verifique a seguinte pergunta considerando o contexto da conversa: '{input}'"
    identification_result = agent_request_identifier.invoke(
        {"input": identify_input},
        config=config,
    )
    return identification_result["output"]

In [ ]:
import datetime

def query_rag(input: str) -> str:

    # Identifica a requisição do usuário
    request = identify_request(input)

    # Se for uma requisição de ajuda, chama o agente de ajuda
    if request == "Ajuda":
        response = agent_helper_with_chat_history.invoke(
            {"input": input},
            config=config,
        )["output"]

        # Verifica se a resposta gerada atende aos critérios
        verification_result = verify_response(input, response)
        
        if "Resposta válida" in verification_result:
            return response
        else:
            return verification_result
        
    # Se for uma requisição de calendário, chama o agente de calendário
    elif request == "Calendário":
        current_datetime = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
        current_weekday = datetime.datetime.now().strftime("%A")
        response = agent_google_calendar_with_chat_history.invoke(
            {"input": input, "current_datetime": current_datetime, "current_weekday": current_weekday},
            config=config,
        )["output"]

        return response

    # Se não for uma requisição válida, retorna uma mensagem de erro
    else:
        return "Desculpe, não consegui entender sua pergunta. Pode reformular?"    


In [ ]:
import gradio as gr

iface = gr.Interface(
        fn=query_rag,
        inputs="text",
        outputs="text",
        title="Groq ChatBot",
        description="Groq ChatBot, o seu bot para facilitar sua vida!",
    )

iface.launch(share=True)